In [1]:
import polars as pl
import plotly.graph_objects as go
import plotly.express as px
import re
import os
import pandas as pd

In [2]:
folder_path = "../data/parquet_files"
files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.parquet')]

data_frames = [pl.read_parquet(file) for file in files]
combined_df = pl.concat(data_frames)

In [3]:
def add_prediction_column(df):
    """Add a prediction column based on the score threshold of 0.5."""
    return df.with_columns((df['score'] > 0.5).cast(pl.Int8).alias('prediction'))

def add_threshold_columns(df, threshold):
    return df.with_columns((df['score'] > threshold).cast(pl.Int8).alias(f'threshold_{str(threshold)}'))

combined_df = add_prediction_column(combined_df)
combined_df = add_threshold_columns(combined_df, 0.5)
combined_df = add_threshold_columns(combined_df, 0.9)

In [4]:
def add_cell_line_run_column(df):
    df = df.with_columns((pl.col("cell_line") + "_" + pl.col("run")).alias("cell_line_run"))
    return df

combined_df = add_cell_line_run_column(combined_df)

In [5]:
print(combined_df.columns)

['cell_line', 'run', 'transcript_id', 'transcript_position', 'sequence', 'proportion_1', 'proportion_2', 'proportion_3', 'diff_1_1', 'diff_1_2', 'diff_1_3', 'diff_2_1', 'diff_2_2', 'diff_2_3', 'length', 'mean_0', 'var_0', 'max_0', 'min_0', 'mean_1', 'var_1', 'max_1', 'min_1', 'mean_2', 'var_2', 'max_2', 'min_2', 'mean_3', 'var_3', 'max_3', 'min_3', 'mean_4', 'var_4', 'max_4', 'min_4', 'mean_5', 'var_5', 'max_5', 'min_5', 'mean_6', 'var_6', 'max_6', 'min_6', 'mean_7', 'var_7', 'max_7', 'min_7', 'mean_8', 'var_8', 'max_8', 'min_8', 'score', 'prediction', 'threshold_0.5', 'threshold_0.9', 'cell_line_run']


In [ ]:

def calculate_prediction_counts_by_cell_line(df, position_range=(0, 250000)):
    # Filter to the specified transcript_position range
    df = df.filter((pl.col("transcript_position") >= position_range[0]) & (pl.col("transcript_position") <= position_range[1]))

    # Calculate counts and proportions of prediction == 1 and prediction == 0 for each cell line
    count_df = (
        df.group_by("cell_line")
        .agg([
            (pl.col("prediction") == 1).sum().alias("count_prediction_1"),  # Count of prediction == 1
            (pl.col("prediction") == 0).sum().alias("count_prediction_0")   # Count of prediction == 0
        ])
        .with_columns([
            (pl.col("count_prediction_1") / (pl.col("count_prediction_1") + pl.col("count_prediction_0"))).alias("proportion_prediction_1"),
            (pl.col("count_prediction_0") / (pl.col("count_prediction_1") + pl.col("count_prediction_0"))).alias("proportion_prediction_0")
        ])
        .sort("cell_line")
    )

    return count_df

def plot_prediction_counts_by_cell_line_bar(count_df):
    count_df_pandas = count_df.to_pandas()

    fig = go.Figure()

    colors = {
        "modifications": "rgb(255, 127, 14)",  # orange for mods
        "non_modifications": "rgb(31, 119, 180)"  # Blue for non-modifications 
    }

    # Add bar for prediction == 1 (modifications) with count and proportion
    fig.add_trace(go.Bar(
        x=count_df_pandas["cell_line"],
        y=count_df_pandas["count_prediction_1"],
        name="Modifications (Prediction=1)",
        marker_color=colors["modifications"],
        text=(count_df_pandas["count_prediction_1"].astype(str) + " (" +
              (count_df_pandas["proportion_prediction_1"] * 100).round(2).astype(str) + '%)'),
        textposition="outside"
    ))

    # Add bar for prediction == 0 (non-modifications) with count and proportion
    fig.add_trace(go.Bar(
        x=count_df_pandas["cell_line"],
        y=count_df_pandas["count_prediction_0"],
        name="Non-Modifications (Prediction=0)",
        marker_color=colors["non_modifications"],
        text=(count_df_pandas["count_prediction_0"].astype(str) + " (" +
              (count_df_pandas["proportion_prediction_0"] * 100).round(2).astype(str) + '%)'),
        textposition="outside"
    ))

    # Update layout to make it a grouped bar chart
    fig.update_layout(
        title={
            "text": "Counts and Proportions of Modifications and Non-Modifications across Cell Lines",
            "xanchor": 'center',
            "x": 0.5
        },
        xaxis_title="Cell Line",
        yaxis_title="Counts",
        barmode="group",  
        legend_title="Modification Type",
        plot_bgcolor="white"  
    )

    fig.show()

count_df = calculate_prediction_counts_by_cell_line(combined_df)
plot_prediction_counts_by_cell_line_bar(count_df)


### Correlations between Cell Lines

Am not too clear on the correlation calculations but it should resolve positions which other lines/runs do not have data on by ignoring it ie. if run A has position 244 but run B does not, I think it ignores

Also there is for some reason duplicate transcript_positions which I am quite confused about

In [46]:
def prepare_correlation_data(df):
    
    # Aggregate by taking the mean prediction for each transcript_position and cell_line_run combination
    # Extra step to handle duplicate positions? - for some reason there are duplicates
    aggregated_df = (
        df.group_by(["transcript_position", "cell_line_run"])
        .agg(pl.col("prediction").mean().alias("mean_prediction"))
    )
    
    pivot_df = (
        aggregated_df.to_pandas()
        .pivot(index="transcript_position", columns="cell_line_run", values="mean_prediction")
        .fillna(0) 
    )

    correlation_matrix = pivot_df.corr()

    return correlation_matrix

def plot_correlation_heatmap(correlation_matrix, maxz, minz):
    fig = go.Figure(
        data=go.Heatmap(
            z=correlation_matrix.values,
            x=correlation_matrix.columns,
            y=correlation_matrix.index,
            colorscale=px.colors.diverging.RdBu,
            colorbar=dict(title="Correlation Coefficient"),
            zmin=minz,
            zmax=maxz # for best effect, set this to 0.4
        )
    )

    fig.update_layout(
        title={
            "text": "Correlation Heatmap of Predictions across Cell Lines",
            "xanchor": 'center',
            "x": 0.5
        },
    )

    fig.show()

correlation_matrix = prepare_correlation_data(combined_df)
plot_correlation_heatmap(correlation_matrix, 1, -1)

In [47]:
plot_correlation_heatmap(correlation_matrix,0.35, 0)

### Prediction x Feature Correlations

Largely similar to the correlation heatmap

In [ ]:
def calculate_prediction_correlations(df):
    numerical_df = df.select([col for col in df.columns if df.schema[col] in [pl.Float64, pl.Int64, pl.Int8]])
    numerical_df = numerical_df.to_pandas()

    prediction_correlations = numerical_df.corr()['prediction'].drop('prediction')  # Drop self-correlation

    return prediction_correlations

def plot_prediction_correlation_heatmap(prediction_correlations):
    fig = go.Figure(
        data=go.Heatmap(
            z=prediction_correlations.values.reshape(1, -1),
            x=prediction_correlations.index,
            y=["prediction"],
            colorscale="Viridis",
            colorbar=dict(title="Correlation Coefficient")
        )
    )

    fig.update_layout(
        title="Correlation of Features with Prediction",
        xaxis_title="Features",
        yaxis_title="Prediction",
        template="plotly_dark"
    )

    fig.show()

prediction_correlations = calculate_prediction_correlations(combined_df)
plot_prediction_correlation_heatmap(prediction_correlations)

### Modification Proportions across the RNA Sequence

3 main distinct positions with the modifications

1) 0 - 30k
2) 50 - 70k
3) 90 - 100k

In [36]:
def calculate_prediction_counts(df, bin_size=1000, position_range=(0, 3000000)):
    # Filter to the specified transcript_position range
    df = df.filter((pl.col("transcript_position") >= position_range[0]) & (pl.col("transcript_position") <= position_range[1]))

    df = df.with_columns(((pl.col("transcript_position") // bin_size) * bin_size).alias("binned_position"))

    # Calculate the counts of prediction == 1 and prediction == 0
    count_df = (
        df.group_by("binned_position")
        .agg([
            (pl.col("prediction") == 1).sum().alias("count_prediction_1"),  # Count of prediction == 1
            (pl.col("prediction") == 0).sum().alias("count_prediction_0")   # Count of prediction == 0
        ])
        .sort("binned_position")
    )

    return count_df

def plot_prediction_counts(count_df):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=count_df["binned_position"],
        y=count_df["count_prediction_1"],
        mode="lines+markers",
        name="Count of Prediction=1",
        line=dict(color="blue")
    ))

    fig.add_trace(go.Scatter(
        x=count_df["binned_position"],
        y=count_df["count_prediction_0"],
        mode="lines+markers",
        name="Count of Prediction=0",
        line=dict(color="orange")
    ))

    fig.update_layout(
        title="Counts of Modifications across Transcript Positions",
        xaxis_title="Binned Transcript Position",
        yaxis_title="Count of Modifications",
        legend_title="Prediction",
    )

    fig.show()

count_df = calculate_prediction_counts(combined_df)
plot_prediction_counts(count_df)

In [ ]:
def calculate_prediction_proportion(df, bin_size=1000, position_range=(0, 250000)):
    # Filter to the specified transcript_position range
    df = df.filter((pl.col("transcript_position") >= position_range[0]) & (pl.col("transcript_position") <= position_range[1]))

    df = df.with_columns(((pl.col("transcript_position") // bin_size) * bin_size).alias("binned_position"))

    proportion_df = (
        df.group_by("binned_position")
        .agg([
            (pl.col("prediction") == 1).mean().alias("proportion_prediction_1")
        ])
        .sort("binned_position")
    )

    return proportion_df

def plot_prediction_proportion(proportion_df):
    fig = go.Figure(
        data=go.Scatter(
            x=proportion_df["binned_position"],
            y=proportion_df["proportion_prediction_1"],
            mode="lines+markers",
            name="Proportion of Prediction=1"
        )
    )

    fig.update_layout(
        title="Proportion of Modifications across Transcript Positions",
        xaxis_title="Binned Transcript Position",
        yaxis_title="Proportion of m6a Modifications",
    )

    fig.show()

# Example Usage
# Assuming combined_df is your Polars DataFrame with 'transcript_position' and 'prediction' columns
proportion_df = calculate_prediction_proportion(combined_df)
plot_prediction_proportion(proportion_df)

In [ ]:
def calculate_prediction_proportion_by_cell_line(df, bin_size=1000, position_range=(0, 150000)):
    df = df.filter((pl.col("transcript_position") >= position_range[0]) & (pl.col("transcript_position") <= position_range[1]))
    df = df.with_columns(((pl.col("transcript_position") // bin_size) * bin_size).alias("binned_position"))

    proportion_df = (
        df.group_by(["cell_line_run", "binned_position"])
        .agg([
            (pl.col("prediction") == 1).mean().alias("proportion_prediction_1")
        ])
        .sort(["cell_line_run", "binned_position"])
    )

    all_bins = pl.DataFrame({
        "binned_position": list(range(position_range[0], position_range[1] + bin_size, bin_size))
    })
    cell_line_runs = df.select("cell_line_run").unique()

    # Cross join to create all possible combinations of cell_line_run and binned_position
    complete_df = cell_line_runs.join(all_bins, how="cross")

    # Join with proportion_df to fill missing bins with 0
    filled_df = complete_df.join(proportion_df, on=["cell_line_run", "binned_position"], how="left").fill_null(0)

    return filled_df

def plot_prediction_proportion_by_cell_line(proportion_df):
    fig = go.Figure()

    # Convert Polars DataFrame to Pandas for Plotly compatibility
    proportion_df_pandas = proportion_df.to_pandas()

    for cell_line_run in proportion_df_pandas['cell_line_run'].unique():
        subset = proportion_df_pandas[proportion_df_pandas['cell_line_run'] == cell_line_run]
        
        fig.add_trace(
            go.Scatter(
                x=subset['binned_position'],
                y=subset['proportion_prediction_1'],
                mode="lines+markers",
                name=cell_line_run
            )
        )

    fig.update_layout(
        title="Proportion of Prediction=1 across Transcript Positions by Cell Line + Run",
        xaxis_title="Binned Transcript Position",
        yaxis_title="Proportion of Prediction=1",
        legend_title="Cell Line + Run",
        template="plotly_dark"
    )

    fig.show()

proportion_df = calculate_prediction_proportion_by_cell_line(combined_df)
plot_prediction_proportion_by_cell_line(proportion_df)

In [7]:
def calculate_prediction_proportion_by_cell_line(df, bin_size=1000, position_range=(0, 150000)):
    df = df.filter((pl.col("transcript_position") >= position_range[0]) & (pl.col("transcript_position") <= position_range[1]))
    df = df.with_columns(((pl.col("transcript_position") // bin_size) * bin_size).alias("binned_position"))

    proportion_df = (
        df.group_by(["cell_line", "binned_position"])
        .agg([
            (pl.col("prediction") == 1).mean().alias("proportion_prediction_1")
        ])
        .sort(["cell_line", "binned_position"])
    )

    all_bins = pl.DataFrame({
        "binned_position": list(range(position_range[0], position_range[1] + bin_size, bin_size))
    })
    cell_line_runs = df.select("cell_line").unique()

    complete_df = cell_line_runs.join(all_bins, how="cross")

    filled_df = complete_df.join(proportion_df, on=["cell_line", "binned_position"], how="left").fill_null(0)

    return filled_df

def plot_prediction_proportion_by_cell_line(proportion_df):
    fig = go.Figure()

    # Convert Polars DataFrame to Pandas for Plotly compatibility
    proportion_df_pandas = proportion_df.to_pandas()

    # Plot each cell_line_run with a different color
    for cell_line_run in proportion_df_pandas['cell_line'].unique():
        subset = proportion_df_pandas[proportion_df_pandas['cell_line'] == cell_line_run]
        
        fig.add_trace(
            go.Scatter(
                x=subset['binned_position'],
                y=subset['proportion_prediction_1'],
                mode="lines+markers",
                name=cell_line_run
            )
        )

    fig.update_layout(
        title="Proportion of Modifications across Transcript Positions by Cell Line",
        xaxis_title="Binned Transcript Position",
        yaxis_title="Proportion of m6a Modifications",
        legend_title="Cell Line + Run",
    )

    fig.show()

proportion_df = calculate_prediction_proportion_by_cell_line(combined_df)
plot_prediction_proportion_by_cell_line(proportion_df)

Making a note here but if you look at MCF7 Cell Line versus the rest, there are 5 distinct positions where there are modifications detected where most other cell lines have 3 or max 4 positions, this corroborates the correlation between cell lines heatmap (the white and blue correlation heatmap)

In [29]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_prediction_proportion_by_cell_line_faceted(proportion_df):
    proportion_df_pandas = proportion_df.to_pandas()
    
    unique_cell_lines = proportion_df_pandas['cell_line'].unique()
    num_rows = len(unique_cell_lines)
    scientific_palette = [
        "rgb(31, 119, 180)",   # Blue
        "rgb(255, 127, 14)",   # Orange
        "rgb(44, 160, 44)",    # Green
        "rgb(214, 39, 40)",    # Red
        "rgb(148, 103, 189)",  # Purple
    ]

    fig = make_subplots(
        rows=num_rows, cols=1, 
        shared_xaxes=True, 
        vertical_spacing=0.05, 
        subplot_titles=[f"Cell Line: {cell_line}" for cell_line in unique_cell_lines]
    )

    for i, cell_line in enumerate(unique_cell_lines, start=1):
        subset = proportion_df_pandas[proportion_df_pandas['cell_line'] == cell_line]
        
        fig.add_trace(
            go.Scatter(
                x=subset['binned_position'],
                y=subset['proportion_prediction_1'],
                mode="lines+markers",
                name=cell_line,
                line=dict(color=scientific_palette[i % len(scientific_palette)]),  # Assign color from palette
                marker=dict(color=scientific_palette[i % len(scientific_palette)])
            ),
            row=i, col=1
        )

        fig.update_yaxes(range=[0, 0.175], row=i, col=1)
        fig.update_xaxes(showticklabels=True, row=i, col=1)

    # Update layout for the overall figure with increased height and shared y-axis title
    fig.update_layout(
        title={
            'text': "Proportion of Modifications across Transcript Positions by Cell Line",
            'x': 0.5,  # Center the title
            'xanchor': 'center'
        },
        yaxis_title={'text': "Proportion of m6a Modifications"},  # Single y-axis title across all subplots
        height=200 * num_rows,  # Adjust height based on the number of rows
        showlegend=False,  # Hides the legend, each cell line is in a separate subplot
    )

    fig.update_xaxes(title_text="Binned Transcript Position", row=num_rows, col=1)
    
    fig.show()

# Call the modified plotting function
plot_prediction_proportion_by_cell_line_faceted(proportion_df)

In [72]:
print(combined_df.columns)

['cell_line', 'run', 'transcript_id', 'transcript_position', 'sequence', 'proportion_1', 'proportion_2', 'proportion_3', 'diff_1_1', 'diff_1_2', 'diff_1_3', 'diff_2_1', 'diff_2_2', 'diff_2_3', 'length', 'mean_0', 'var_0', 'max_0', 'min_0', 'mean_1', 'var_1', 'max_1', 'min_1', 'mean_2', 'var_2', 'max_2', 'min_2', 'mean_3', 'var_3', 'max_3', 'min_3', 'mean_4', 'var_4', 'max_4', 'min_4', 'mean_5', 'var_5', 'max_5', 'min_5', 'mean_6', 'var_6', 'max_6', 'min_6', 'mean_7', 'var_7', 'max_7', 'min_7', 'mean_8', 'var_8', 'max_8', 'min_8', 'score', 'prediction', 'threshold_0.5', 'threshold_0.9', 'cell_line_run']


In [8]:
def find_common_transcript_ids(df):
    common_transcripts = (
        df.group_by("transcript_id")
        .agg([
            pl.col("cell_line").n_unique().alias("cell_line_count"),  
            pl.col("cell_line").unique().alias("unique_cell_lines")   
        ])
        .filter(pl.col("cell_line_count") > 1) 
    )

    return common_transcripts.select(["transcript_id", "unique_cell_lines"])

common_transcripts = find_common_transcript_ids(combined_df)
print(common_transcripts)


shape: (59_354, 2)
┌─────────────────┬───────────────────────────────┐
│ transcript_id   ┆ unique_cell_lines             │
│ ---             ┆ ---                           │
│ str             ┆ list[str]                     │
╞═════════════════╪═══════════════════════════════╡
│ ENST00000565873 ┆ ["A549", "MCF7", … "K562"]    │
│ ENST00000509437 ┆ ["MCF7", "K562", … "A549"]    │
│ ENST00000309060 ┆ ["Hct116", "HepG2", … "A549"] │
│ ENST00000529422 ┆ ["MCF7", "HepG2", … "Hct116"] │
│ ENST00000397961 ┆ ["HepG2", "A549", … "K562"]   │
│ …               ┆ …                             │
│ ENST00000374867 ┆ ["Hct116", "HepG2", "A549"]   │
│ ENST00000555235 ┆ ["HepG2", "Hct116", … "A549"] │
│ ENST00000546959 ┆ ["A549", "K562", … "Hct116"]  │
│ ENST00000372003 ┆ ["HepG2", "Hct116", … "MCF7"] │
│ ENST00000501280 ┆ ["Hct116", "K562", "HepG2"]   │
└─────────────────┴───────────────────────────────┘


In [11]:
common_transcripts = find_common_transcript_ids(combined_df)
filtered_df = combined_df.join(common_transcripts, on="transcript_id", how="inner")

proportion_df = calculate_prediction_proportion_by_cell_line(filtered_df)
plot_prediction_proportion_by_cell_line(proportion_df)